<img src="http://www.espol.edu.ec/sites/default/files/nuevaespol/logo_negro.png" width="301" height="58" />

#     FACULTAD DE INGENIER&Iacute;A EN ELECTRICIDAD Y COMPUTACI&Oacute;N
##   DIPLOMADO EN CIENCIA DE DATOS PARA LOS NEGOCIOS
###  Laboratorio 5: Docker & Spark

<h3><strong>Introducci&oacute;n</strong></h3>

Docker es un proyecto de código abierto que automatiza el despliegue de aplicaciones dentro de contenedores de software, proporcionando una capa adicional de abstracción y automatización de virtualización de aplicaciones en múltiples sistemas operativos.

Apache Spark es un framework de computación en clúster open-source. Fue desarrollada originariamente en la Universidad de California, en el AMPLab de Berkeley. El código base del proyecto Spark fue donado más tarde a la Apache Software Foundation que se encarga de su mantenimiento desde entonces. 

<h3>Procedimiento</h3>

<h3>Paso 1: Instalación de contenedores</h3>

#Instalación de Jupyter

docker run -ti -p 8888:8888 emanuelfontelles/spark-jupyter

#Instalación de pyspark dentro del contenedor

pip install pyspark #Trabajos con Python

<h3>Paso 2: Comprobación de pyspark</h3>

Un RDD es una estructura de datos paralela que distribuye la carga de trabajo entre los nodos de trabajo. Son las unidades básicas de programación de Spark.
Para trabajar con RDD, necesitamos crear un SparkContext.

In [2]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

Un SparkContext es la puerta de entrada para el entorno Spark. Para cada SparkApp necesita crear el objeto SparkContext. Permite que su aplicación Spark acceda a Spark Cluster con la ayuda de Resource Manager (Spark Standalone / Yarn / Mesos). Para crear SparkContext, se debe hacer el primer SparkConf. SparkConf tiene un parámetro de configuración que nuestra aplicación de controlador de Spark pasará a SparkContext.

En Spark 2 puede usar SparkSession en lugar de SparkContext. En Spark 2.0 en adelante, es mejor usar SparkSession ya que proporciona acceso a todas las funcionalidades de Spark que proporciona SparkContext. Además, proporciona API para trabajar con DataFrames y DataSets.

El siguiente código es útil cuando desea crear una SparkSession;

In [3]:
spark=SparkSession.builder.master("local[*]").appName("WordCount").getOrCreate()

Si escribimos local [*] en el maestro, usará todos los núcleos en nuestra máquina. Si dijimos local [4] funcionará con 4 núcleos. Puede funcionar en modo local / hilo / mesos y kubernetes.

getOrCreate se usa para verificar si existe una SparkSession. Crea uno nuevo si no.

Si no importamos SparkConf, tendremos que asignar métricas complejas para principiantes, como los valores "executeor.memory" y "driver.memory". En tales casos, puede determinar los valores de configuración como en el siguiente ejemplo.

In [4]:
sc=spark.sparkContext

Leemos una archivo

In [33]:
location="/home/user/notebooks/data.txt"
data_rdd=sc.textFile(location)
data_rdd.take(100)

['The Project Gutenberg EBook of Romeo and Juliet, by William Shakespeare',
 '',
 '',
 '*******************************************************************',
 "THIS EBOOK WAS ONE OF PROJECT GUTENBERG'S EARLY FILES PRODUCED AT A",
 'TIME WHEN PROOFING METHODS AND TOOLS WERE NOT WELL DEVELOPED. THERE',
 'IS AN IMPROVED EDITION OF THIS TITLE WHICH MAY BE VIEWED AS EBOOK',
 '(#100) at https://www.gutenberg.org/ebooks/100',
 '*******************************************************************',
 '',
 '',
 'This eBook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever.  You may copy it, give it away or',
 're-use it under the terms of the Project Gutenberg License included',
 'with this eBook or online at www.gutenberg.org/license',
 '',
 '',
 'Title: Romeo and Juliet',
 '',
 'Author: William Shakespeare',
 '',
 'Posting Date: May 25, 2012 [EBook #1112]',
 'Release Date: November, 1997  [Etext #1112]',
 '',
 'Language: English',
 '',
 'Character set e

<strong>Eliminar puntuación y transformar todas las palabras en minúsculas</strong>

Para excluir los valores de puntuación y convertir todas las palabras en minúsculas, escribimos una función como la siguiente.

In [34]:
def lower_clean_str(x):
    punc=':"#$%&\'()*+,./:;<=>?%[\\]^_`{|}~-'
    lowercased_str = x.lower()
    for ch in punc:
        lowercased_str = lowercased_str.replace(ch, '')
    return lowercased_str

data_rdd=data_rdd.map(lower_clean_str)
data_rdd.take(50)

['the project gutenberg ebook of romeo and juliet by william shakespeare',
 '',
 '',
 '',
 'this ebook was one of project gutenbergs early files produced at a',
 'time when proofing methods and tools were not well developed there',
 'is an improved edition of this title which may be viewed as ebook',
 '100 at httpswwwgutenbergorgebooks100',
 '',
 '',
 '',
 'this ebook is for the use of anyone anywhere at no cost and with',
 'almost no restrictions whatsoever  you may copy it give it away or',
 'reuse it under the terms of the project gutenberg license included',
 'with this ebook or online at wwwgutenbergorglicense',
 '',
 '',
 'title romeo and juliet',
 '',
 'author william shakespeare',
 '',
 'posting date may 25 2012 ebook 1112',
 'release date november 1997  etext 1112',
 '',
 'language english',
 '',
 'character set encoding ascii',
 '',
 ' start of this project gutenberg ebook romeo and juliet ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'project gutenberg

<strong>Divide la oración en una lista de palabras</strong>

La función split () divide la cadena en el parámetro dado y estamos dividiéndonos por el carácter de espacio.
Usamos la función de división para separar las palabras en todas las líneas.

In [35]:
data_rdd=data_rdd.flatMap(lambda satir: satir.split(" "))
data_rdd.take(5)

['the', 'project', 'gutenberg', 'ebook', 'of']

<strong>Excluir espacios en blanco</strong>

Realizamos un filtrado a continuación para excluir espacios en blanco.

In [36]:
data_rdd=data_rdd.filter(lambda x:x!='')
data_rdd.take(10)

['the',
 'project',
 'gutenberg',
 'ebook',
 'of',
 'romeo',
 'and',
 'juliet',
 'by',
 'william']

<strong>Cuenta cuántas veces aparece cada palabra</strong>

Para hacer este cálculo, podemos aplicar la transformación "reduceByKey" en el par (clave, val) RDD.

Para usar la transformación "reduceByKey" para encontrar las frecuencias de cada palabra, en primer lugar, se requiere un par (clave, val) RDD; En este par (clave, val) RDD, clave es la palabra y val es 1 para cada palabra en RDD (1 representa el número de cada palabra en "data_rdd").

Al principio, necesitamos convertir "data_rdd" en par (clave, val) RDD (data_count)

In [37]:
data_count=data_rdd.map(lambda word:(word,1))
data_count.take(10)

[('the', 1),
 ('project', 1),
 ('gutenberg', 1),
 ('ebook', 1),
 ('of', 1),
 ('romeo', 1),
 ('and', 1),
 ('juliet', 1),
 ('by', 1),
 ('william', 1)]

In [38]:
data_count_rbk=data_count.reduceByKey(lambda x,y:(x+y)).sortByKey()
data_count_rbk.take(40)

[('1', 16),
 ('100', 1),
 ('1112', 2),
 ('1112txt', 1),
 ('1112zip', 1),
 ('1500', 1),
 ('1595', 1),
 ('19901993', 1),
 ('1997', 1),
 ('1a', 1),
 ('1b', 1),
 ('1c', 2),
 ('1d', 1),
 ('1e', 2),
 ('1e1', 5),
 ('1e2', 1),
 ('1e3', 1),
 ('1e4', 1),
 ('1e5', 1),
 ('1e6', 1),
 ('1e7', 3),
 ('1e8', 4),
 ('1e9', 3),
 ('1f', 1),
 ('1f1', 1),
 ('1f2', 1),
 ('1f3', 5),
 ('1f4', 1),
 ('1f5', 1),
 ('1f6', 1),
 ('2', 10),
 ('20', 1),
 ('2001', 1),
 ('2012', 1),
 ('25', 1),
 ('3', 6),
 ('30', 1),
 ('4', 3),
 ('4557', 1),
 ('5', 1)]

<strong>Clasifica las palabras</strong>

Queremos ordenar las palabras más frecuentes en orden descendente. Como primer paso, cambiamos los pares (clave, val) como (val, clave).

In [41]:
data_count_rbk=data_count_rbk.map(lambda x:(x[1],x[0]))
data_count_rbk.take(10)

[(16, '1'),
 (1, '100'),
 (2, '1112'),
 (1, '1112txt'),
 (1, '1112zip'),
 (1, '1500'),
 (1, '1595'),
 (1, '19901993'),
 (1, '1997'),
 (1, '1a')]

In [42]:
data_count_rbk.sortByKey(False).take(10)

[(866, 'the'),
 (793, 'and'),
 (621, 'to'),
 (584, 'i'),
 (535, 'of'),
 (527, 'a'),
 (377, 'in'),
 (373, 'is'),
 (363, 'that'),
 (360, 'my')]

<strong>Excluir StopWords</strong>

Para excluir palabras vacías, descargamos la biblioteca nltk y obtenemos la lista de palabras vacías en inglés.

In [44]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [46]:
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

Cuando excluimos valores de palabras vacías, vemos que la palabra "thou" es la palabra más común.

In [49]:
data_count_rbk=data_count_rbk.filter(lambda x: x[1] not in stopwords).sortByKey(False)
data_count_rbk.sortByKey(False).take(20)

[(276, 'thou'),
 (164, 'thy'),
 (163, 'rom'),
 (147, 'nurse'),
 (134, 'thee'),
 (131, 'love'),
 (128, 'romeo'),
 (117, 'jul'),
 (112, 'shall'),
 (97, 'come'),
 (91, 'friar'),
 (90, 'project'),
 (83, 'ill'),
 (82, 'enter'),
 (82, 'good'),
 (74, 'go'),
 (70, 'well'),
 (68, 'man'),
 (66, 'death'),
 (66, 'may')]

# Bibliografía
<ul>
<li><a href="https://www.docker.com">https://www.docker.com</a></li>
<li><a href="https://spark.apache.org">https://spark.apache.org</a></li>
    <li><a href="https://spark.apache.org/docs/latest/api/python/index.html">https://spark.apache.org/docs/latest/api/python/index.html</a></li>
</ul>